In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
#import tensorflow_text as text
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.models import load_model, Sequential
from keras.layers import Embedding, Dense, SimpleRNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import string
import re

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, \
    Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

## Hypothesis 3

In [87]:
class Model3():

    def __init__(self):
        pass

    def create_model(self, ref_lang, tran_lang, max_length_inp, latent_dim=300, embedding_dim=100):

        self.embeddings_index = {}
        f = open('glove.6B.100d.txt', encoding='utf-8')
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            self.embeddings_index[word] = coefs
        f.close()

        self.latent_dim = latent_dim
        self.embedding_dim = embedding_dim

        # Encoder
        self.encoder_inputs = Input(shape=(max_length_inp, ))

        embeddings_matrix = self.get_embeddings(ref_lang)

        # Embedding layer
        self.enc_emb = Embedding(len(ref_lang.word_index) + 1, embedding_dim, input_length=max_length_inp, weights=[embeddings_matrix],
                              trainable=False)(self.encoder_inputs)


        # Encoder LSTM 1
        self.encoder_lstm1 = LSTM(latent_dim, return_sequences=True,
                              return_state=True, dropout=0.4,
                              recurrent_dropout=0.4)
        (self.encoder_output, self.state_h, self.state_c) = self.encoder_lstm1(self.enc_emb)

        # Set up the decoder, using encoder_states as the initial state
        self.decoder_inputs = Input(shape=(None, ))

        embeddings_matrix = self.get_embeddings(tran_lang)
        # Embedding layer
        self.dec_emb = Embedding(len(tran_lang.word_index) + 1, embedding_dim, weights=[embeddings_matrix],
                              trainable=False)(self.decoder_inputs)

        # Decoder LSTM
        self.decoder_lstm = LSTM(latent_dim, return_sequences=True,
                              return_state=True, dropout=0.4,
                              recurrent_dropout=0.2)
        (decoder_outputs, decoder_fwd_state, decoder_back_state) = \
              self.decoder_lstm(self.dec_emb, initial_state=[self.state_h, self.state_c])

        # Dense layer
        self.decoder_dense = TimeDistributed(Dense(len(tran_lang.word_index) + 1, activation='softmax'))
        decoder_outputs = self.decoder_dense(decoder_outputs)

        # Define the model
        return Model([self.encoder_inputs, self.decoder_inputs], decoder_outputs)

    def get_embeddings(self, lang):
        word_index = lang.word_index
        vocab_size = len(word_index)

        embeddings_matrix = np.zeros((vocab_size+1, self.embedding_dim))
        for word, i in word_index.items():
            embedding_vector = self.embeddings_index.get(word)
            if embedding_vector is not None:
                embeddings_matrix[i] = embedding_vector

        return embeddings_matrix

    def inference_model(self):

        # Encode the input sequence to get the feature vector
        encoder_model = Model(inputs=self.encoder_inputs, outputs=[self.encoder_output,
                                self.state_h, self.state_c])

        # Below tensors will hold the states of the previous time step
        decoder_state_input_h = Input(shape=(self.latent_dim, ))
        decoder_state_input_c = Input(shape=(self.latent_dim, ))
        decoder_hidden_state_input = Input(shape=(max_length_inp, self.latent_dim))

        # Get the embeddings of the decoder sequence
        dec_emb2 = self.dec_emb

        # To predict the next word in the sequence, set the initial states to the states from the previous time step
        (decoder_outputs2, state_h2, state_c2) = self.decoder_lstm(dec_emb2,
                  initial_state=[decoder_state_input_h, decoder_state_input_c])

        # A dense softmax layer to generate prob dist. over the target vocabulary
        decoder_outputs2 = self.decoder_dense(decoder_outputs2)

        # Final decoder model
        decoder_model = Model([self.decoder_inputs] + [decoder_hidden_state_input,
                                decoder_state_input_h, decoder_state_input_c],
                                [decoder_outputs2] + [state_h2, state_c2])

        return encoder_model, decoder_model

In [ ]:
def predict_sequence(input_seq, encoder_model, decoder_model):
    reverse_target_word_index = tran_lang.index_word
    reverse_source_word_index = ref_lang.index_word
    target_word_index = tran_lang.word_index

    # Encode the input as state vectors.
    (e_out, e_h, e_c) = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1
    target_seq = np.zeros((1, 1))

    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sos']

    stop_condition = False
    decoded_sentence = ''

    while not stop_condition:
        (output_tokens, h, c) = decoder_model.predict([target_seq]
                + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index+1]

        if sampled_token != 'eos':
            decoded_sentence += ' ' + sampled_token

        # Exit condition: either hit max length or find the stop word.
        if sampled_token == 'eos' or len(decoded_sentence.split()) >= max_length_targ - 1:
            stop_condition = True

        # Update the target sequence (of length 1)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        (e_h, e_c) = (h, c)

    return decoded_sentence

In [88]:
model3 = Model3()
m3 = model3.create_model(ref_lang, tran_lang, max_length_inp)
m3.summary()

Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_51 (InputLayer)          [(None, 180)]        0           []                               
                                                                                                  
 input_52 (InputLayer)          [(None, None)]       0           []                               
                                                                                                  
 embedding_19 (Embedding)       (None, 180, 100)     1156100     ['input_51[0][0]']               
                                                                                                  
 embedding_20 (Embedding)       (None, None, 100)    945400      ['input_52[0][0]']               
                                                                                           

In [89]:
m3.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=2)

In [67]:
history = m3.fit(
    [reference_tensor_train, translation_tensor_train[:, :-1]],
    translation_tensor_train.reshape(translation_tensor_train.shape[0], translation_tensor_train.shape[1], 1)[:, 1:],
    epochs=10,
    callbacks=[es],
    batch_size=64,
    validation_data=([reference_tensor_val, translation_tensor_val[:, :-1]],
                     translation_tensor_val.reshape(translation_tensor_val.shape[0], translation_tensor_val.shape[1], 1)[:, 1:]),
    )

Epoch 1/10
125/125 [==============================] - 687s 5s/step - loss: 0.7192 - val_loss: 0.4084
Epoch 2/10
125/125 [==============================] - 685s 5s/step - loss: 0.3964 - val_loss: 0.3950
Epoch 3/10
125/125 [==============================] - 623s 5s/step - loss: 0.4081 - val_loss: 0.3873
Epoch 4/10
125/125 [==============================] - 619s 5s/step - loss: 0.3742 - val_loss: 0.3791
Epoch 5/10
125/125 [==============================] - 611s 5s/step - loss: 0.3647 - val_loss: 0.3725
Epoch 6/10
125/125 [==============================] - 615s 5s/step - loss: 0.3574 - val_loss: 0.3681
Epoch 7/10
125/125 [==============================] - 616s 5s/step - loss: 0.3982 - val_loss: 0.3641
Epoch 8/10
125/125 [==============================] - 626s 5s/step - loss: 0.3470 - val_loss: 0.3622
Epoch 9/10
125/125 [==============================] - 614s 5s/step - loss: 0.3445 - val_loss: 0.3604
Epoch 10/10
125/125 [==============================] - 633s 5s/step - loss: 0.3383 - val_lo

In [68]:
m3.save('model3.h5')

In [90]:
encoder_model3, decoder_model3 = model3.inference_model()

## Prediction

In [91]:
for i in range(0, 5):
    print(predict_sequence(reference_tensor_train[i].reshape(1,max_length_inp), encoder_model3, decoder_model3))

1/1 [==============================] - 0s 38ms/step
 mud mud competence bradley betting mushroom smell crista bloodlust pointed skipping two onto paddle employees creatures stole homeowners homeowners homeowners homeowners homeowners able carolina t homeowners homeowners homeowners homeowners homeowners homeowners sanchez prison tanis seducer bought 'cuckoo' wallets venom vatican vatican steven steven delacroix's jijo jijo badly badly francisco disability disability disability box homeowners homeowners homeowners rosie hovna vo whereas incoming mike onto unpredictable candi candi candi salvadorans salvadorans employees salvadorans staros 'cuckoo' onto onto unpredictable candi candi candi salvadorans salvadorans employees salvadorans staros 'cuckoo' fooling haruna jewish jewish jewish antique club jewish osborne osborne osborne dissension dungeons savages counting confessed confessed fresh efficient efficient addicted homeowners homeowners homeowners homeowners homeowners homeowners hom

1/1 [==============================] - 0s 40ms/step
 mud mud competence bradley betting mushroom smell crista bloodlust pointed skipping two onto paddle employees creatures stole homeowners homeowners homeowners homeowners homeowners able carolina t homeowners homeowners homeowners homeowners homeowners homeowners sanchez prison tanis seducer bought 'cuckoo' wallets venom vatican vatican steven steven delacroix's jijo jijo badly badly francisco disability disability disability box homeowners homeowners homeowners rosie hovna vo whereas incoming mike onto unpredictable candi candi candi salvadorans salvadorans employees salvadorans staros 'cuckoo' onto onto unpredictable candi candi candi salvadorans salvadorans employees salvadorans staros 'cuckoo' fooling haruna jewish jewish jewish antique club jewish osborne osborne osborne dissension dungeons savages counting confessed confessed fresh efficient efficient addicted homeowners homeowners homeowners homeowners homeowners homeowners hom

1/1 [==============================] - 0s 40ms/step
 mud mud competence bradley betting mushroom smell crista bloodlust pointed skipping two onto paddle employees creatures stole homeowners homeowners homeowners homeowners homeowners able carolina t homeowners homeowners homeowners homeowners homeowners homeowners sanchez prison tanis seducer bought 'cuckoo' wallets venom vatican vatican steven steven delacroix's jijo jijo badly badly francisco disability disability disability box homeowners homeowners homeowners rosie hovna vo whereas incoming mike onto unpredictable candi candi candi salvadorans salvadorans employees salvadorans staros 'cuckoo' onto onto unpredictable candi candi candi salvadorans salvadorans employees salvadorans staros 'cuckoo' fooling haruna jewish jewish jewish antique club jewish osborne osborne osborne dissension dungeons savages counting confessed confessed fresh efficient efficient addicted homeowners homeowners homeowners homeowners homeowners homeowners hom